# Enterprise Data Science_COVID-19

### Author: Akanksha Parashar
### Matrikelnummer: 423022

## Notebook Description
- In this notebook, we are trying to create a 'Relational Data Model'.
- The dataset of the COVID infected population is taken from John Hopkins COVID-19 GitHub page.
- The datapath given in this notebook is of user's computer folder where the required csv files are saved.
- The data source is given in the Readme file.

In [21]:
# importing the libraries
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

# Relational data model - defining a primary key
In the relational model of databases, a primary key is a specific choice of a minimal set of attributes (columns) that uniquely specify a tuple (row) in a relation (table) (source: wiki)

A primary key’s main features are:

- It must contain a unique value for each row of data.
- It cannot contain null values.

# Johns Hopkins GITHUB csv data

In [22]:
data_path='C:/Users/hp/Documents/ads_covid_19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)
pd_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/7/22,6/8/22,6/9/22,6/10/22,6/11/22,6/12/22,6/13/22,6/14/22,6/15/22,6/16/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,180741,180784,180864,180864,180864,180864,181120,181178,181236,181465
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,276468,276518,276583,276638,276690,276731,276731,276821,276821,276821
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,265904,265909,265920,265925,265925,265927,265937,265943,265952,265964
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,43067,43224,43224,43224,43224,43224,43224,43224,43449,43449
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,99761,99761,99761,99761,99761,99761,99761,99761,99761,99761


In [23]:
pd_data_base=pd_raw.rename(columns={'Country/Region':'country',
                      'Province/State':'state'})
pd_data_base['state']=pd_data_base['state'].fillna('no')  #ensure a sring, important for indexing

In [24]:
pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)
pd_data_base.head()

,state,country,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/7/22,6/8/22,6/9/22,6/10/22,6/11/22,6/12/22,6/13/22,6/14/22,6/15/22,6/16/22
0,no,Afghanistan,0,0,0,0,0,0,0,0,...,180741,180784,180864,180864,180864,180864,181120,181178,181236,181465
1,no,Albania,0,0,0,0,0,0,0,0,...,276468,276518,276583,276638,276690,276731,276731,276821,276821,276821
2,no,Algeria,0,0,0,0,0,0,0,0,...,265904,265909,265920,265925,265925,265927,265937,265943,265952,265964
3,no,Andorra,0,0,0,0,0,0,0,0,...,43067,43224,43224,43224,43224,43224,43224,43224,43449,43449
4,no,Angola,0,0,0,0,0,0,0,0,...,99761,99761,99761,99761,99761,99761,99761,99761,99761,99761


In [25]:
pd_relational_model=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )
pd_relational_model.head()

,date,state,country,confirmed
0,1/22/20,Alberta,Canada,0.0
1,1/22/20,Anguilla,United Kingdom,0.0
2,1/22/20,Anhui,China,1.0
3,1/22/20,Aruba,Netherlands,0.0
4,1/22/20,Australian Capital Territory,Australia,0.0


In [26]:
pd_relational_model.dtypes

date          object
state         object
country       object
confirmed    float64
dtype: object

In [27]:
pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')

In [28]:
pd_relational_model.dtypes

date         datetime64[ns]
state                object
country              object
confirmed           float64
dtype: object

In [29]:
pd_relational_model.confirmed=pd_relational_model.confirmed.astype(int)

pd_relational_model.to_csv('C:/Users/hp/Documents/ads_covid_19/data/processed/COVID_relational_confirmed.csv',sep=';',index=False)

In [30]:
print(pd_relational_model[pd_relational_model['country']=='US'].tail())

             date state country  confirmed
248790 2022-06-12    no      US   85515529
249075 2022-06-13    no      US   85666440
249360 2022-06-14    no      US   85758512
249645 2022-06-15    no      US   85941290
249930 2022-06-16    no      US   86057735


# Attention due to the new structure of the Johns Hopkins data set the state information US is in a second file

In [31]:
data_path='C:/Users/hp/Documents/ads_covid_19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
pd_raw_US=pd.read_csv(data_path)
pd_raw_US.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/7/22,6/8/22,6/9/22,6/10/22,6/11/22,6/12/22,6/13/22,6/14/22,6/15/22,6/16/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,16052,16065,16084,16095,16095,16095,16134,16158,16174,16193
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,56955,57024,57079,57166,57166,57166,57366,57453,57554,57637
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,5733,5734,5744,5748,5748,5748,5757,5764,5769,5771
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,6535,6540,6544,6547,6547,6547,6565,6573,6583,6596
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,15112,15115,15133,15137,15137,15137,15150,15174,15177,15188


In [32]:
# drop all non relevant fields! 
# Many of this fields do not belong to 
# a transactional data. These fields belong to so called master data
pd_raw_US=pd_raw_US.drop(['UID', 'iso2', 'iso3', 'code3', 'Country_Region','FIPS', 'Admin2',  'Lat', 'Long_', 'Combined_Key'],axis=1)
pd_data_base_US=pd_raw_US.rename(columns={'Province_State':'state'}).copy()

In [33]:
## the stack command has problems when only one information is available within a stacking level
pd_relational_model_US=pd_data_base_US.set_index(['state']) \
                                .T                              \
                                .stack()             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )
pd_relational_model_US['country']='US'
pd_relational_model_US['date']=[datetime.strptime( each,"%m/%d/%y") for each in pd_relational_model_US.date] # convert to datetime
pd_relational_model_US.head()

,date,state,confirmed,country
0,2020-01-22,Alabama,0,US
1,2020-01-22,Alabama,0,US
2,2020-01-22,Alabama,0,US
3,2020-01-22,Alabama,0,US
4,2020-01-22,Alabama,0,US


In [34]:
# update the large relational file

In [35]:
pd_relational_model_all=pd_relational_model[pd_relational_model['country']!='US'].reset_index(drop=True)

In [36]:
pd_relational_model_all=pd.concat([pd_relational_model_all,pd_relational_model_US],ignore_index=True)

In [37]:
pd_relational_model_all.to_csv('C:/Users/hp/Documents/ads_covid_19/data/processed/20200424_COVID_relational_confirmed.csv',sep=';',index=False)

In [38]:
print(pd_relational_model_all[pd_relational_model_all['country']=='US'])

              date    state country  confirmed
249068  2020-01-22  Alabama      US          0
249069  2020-01-22  Alabama      US          0
249070  2020-01-22  Alabama      US          0
249071  2020-01-22  Alabama      US          0
249072  2020-01-22  Alabama      US          0
...            ...      ...     ...        ...
3179997 2022-06-16  Wyoming      US      10554
3179998 2022-06-16  Wyoming      US       5753
3179999 2022-06-16  Wyoming      US          0
3180000 2022-06-16  Wyoming      US       2471
3180001 2022-06-16  Wyoming      US       1632

[2930934 rows x 4 columns]


# cross check on full sum US data

In [39]:
check_US=pd_relational_model_all[pd_relational_model_all['country']=='US']

In [40]:
check_US[['date','country','confirmed']].groupby(['date','country']).sum()

,,confirmed
date,country,
2020-01-22,US,1
2020-01-23,US,1
2020-01-24,US,2
2020-01-25,US,2
2020-01-26,US,5
...,...,...
2022-06-12,US,85515529
2022-06-13,US,85666440
2022-06-14,US,85758512
